In [50]:
import pickle
import pandas as pd
import sys
import numpy as np

In [51]:
sys.path.append("../../code/")
sys.path.append("../code/")


from common_functions import tableau20blind, format_axes, latexify
from regional_average_contribution import contribution

In [52]:
out_overall = pickle.load(open('../../data/input/all_regions.pkl', 'r'))

In [53]:
mf_pred = pickle.load(open('../data/new_mf_pred_all_homes.pkl','r'))
knn_results = pickle.load(open('../data/knn_pred_all_homes.pkl','r'))

In [54]:
from sklearn.metrics import mean_squared_error

In [55]:
thresholds = {'fridge':10,'hvac':100,'dw':0.5,'mw':0.5,'oven':0.5,'wm':0.5}

In [56]:
def compute_rmse_fraction(appliance, pred_df):
    region = "Austin"
    df = out_overall[region]
    #pred_df = mf_pred[appliance][appliance_feature][latent_factors]
    gt_df = df[pred_df.columns].ix[pred_df.index]
    
    if appliance=="hvac":
        start, stop=5,11
    else:
        start, stop=1,13
    aggregate_df = df.ix[pred_df.index][['aggregate_%d' %month for month in range(start, stop)]]

    aggregate_df.columns = gt_df.columns
    
    # Ignoring homes which have some errors
    if appliance in thresholds:
        a = gt_df[gt_df<thresholds[appliance]].sum(axis=1)
        homes_ignore = a[a>0].index
        gt_df_index = [x for x in gt_df.index if x not in homes_ignore]
        gt_df = gt_df.ix[gt_df_index]
    
    a=(aggregate_df<300).sum(axis=1)
    homes_agrgegate_ignore = a[a>0]
    gt_df_index = [x for x in gt_df.index if x not in homes_agrgegate_ignore]
    gt_df = gt_df.ix[gt_df_index]
    
    
    gt_fraction = gt_df.div(aggregate_df)*100
    pred_fraction = pred_df.div(aggregate_df)*100
    
    pred_fraction = pred_fraction.dropna()
    gt_fraction = gt_fraction.dropna()
    common_index = np.intersect1d(gt_fraction.index, pred_fraction.index)
    pred_fraction = pred_fraction.ix[common_index]
    gt_fraction = gt_fraction.ix[common_index]
    
    #print (gt_fraction>1000).sum().sum()
    #e = (pred_fraction>1000).sum(axis=1)
    #print e[e>0]

    rms = np.sqrt(mean_squared_error(pred_fraction,gt_fraction))
    return rms

In [57]:
"""
def compute_rmse_fraction(appliance, pred_df):
    region = "Austin"
    df = out_overall[region]
    #pred_df = mf_pred[appliance][appliance_feature][latent_factors]
    gt_df = df[pred_df.columns].ix[pred_df.index]
    if appliance=="hvac":
        start, stop=5,11
    else:
        start, stop=1,13
    aggregate_df = df.ix[pred_df.index][['aggregate_%d' %month for month in range(start, stop)]]
    
    # Ignoring homes which have some errors
    if appliance in thresholds:
        a = gt_df[gt_df<thresholds[appliance]].sum(axis=1)
        homes_ignore = a[a>0].index
        gt_df_index = [x for x in gt_df.index if x not in homes_ignore]
        gt_df = gt_df.ix[gt_df_index]
    
    a=(aggregate_df<300).sum(axis=1)
    homes_agrgegate_ignore = a[a>0]
    gt_df_index = [x for x in gt_df.index if x not in homes_agrgegate_ignore]
    gt_df = gt_df.ix[gt_df_index]
    

    aggregate_df.columns = gt_df.columns
    gt_fraction = gt_df.div(aggregate_df)*100
    pred_fraction = pred_df.div(aggregate_df)*100
    
    pred_df = pred_df.dropna()
    gt_df = gt_df.dropna()
    common_index = np.intersect1d(gt_df.index, pred_df.index)
    pred_df = pred_df.ix[common_index]
    gt_df = gt_df.ix[common_index]


    rms = np.sqrt(mean_squared_error(pred_df,gt_df))
    return rms
"""


'\ndef compute_rmse_fraction(appliance, pred_df):\n    region = "Austin"\n    df = out_overall[region]\n    #pred_df = mf_pred[appliance][appliance_feature][latent_factors]\n    gt_df = df[pred_df.columns].ix[pred_df.index]\n    if appliance=="hvac":\n        start, stop=5,11\n    else:\n        start, stop=1,13\n    aggregate_df = df.ix[pred_df.index][[\'aggregate_%d\' %month for month in range(start, stop)]]\n    \n    # Ignoring homes which have some errors\n    if appliance in thresholds:\n        a = gt_df[gt_df<thresholds[appliance]].sum(axis=1)\n        homes_ignore = a[a>0].index\n        gt_df_index = [x for x in gt_df.index if x not in homes_ignore]\n        gt_df = gt_df.ix[gt_df_index]\n    \n    a=(aggregate_df<300).sum(axis=1)\n    homes_agrgegate_ignore = a[a>0]\n    gt_df_index = [x for x in gt_df.index if x not in homes_agrgegate_ignore]\n    gt_df = gt_df.ix[gt_df_index]\n    \n\n    aggregate_df.columns = gt_df.columns\n    gt_fraction = gt_df.div(aggregate_df)*100\n

In [58]:
#result = {'MF-105 homes, feature':{},'MF-516 homes':{},'MF-516 homes, feature':{},'KNN':{},'MF-105 homes':{},'FHMM':{},'LBM':{},"RA":{}}
result = {'MF':{},'KNN':{},'FHMM':{},'LBM':{},"RA":{}}

## Computing for KNN

In [59]:
for appliance, appliance_df in knn_results.iteritems():
    
    pred_df = appliance_df
    if appliance=="hvac":
        start, stop=5, 11
    else:
        start, stop=1, 13
    pred_df.columns = [appliance+"_"+str(x) for x in range(start, stop)]
    print pred_df.isnull().sum().sum(), appliance
    rmse_fraction = compute_rmse_fraction(appliance, pred_df)
    

    #rms = np.sqrt(mean_squared_error(gt_df, pred_df))
    result['KNN'][appliance]=rmse_fraction

0 hvac
0 wm
0 fridge
0 mw
0 oven
0 dw


In [60]:
pred_df.median().mean()

9.2638730722984

In [61]:
best_combination = {}
mf_pred_full = pickle.load(open('../data/mf_pred_all_homes.pkl','r'))
for appliance, appliance_struct in mf_pred_full.iteritems():
    print appliance
    best_appliance=1000
    for appliance_feature, appliance_feature_struct in appliance_struct.iteritems():
        for latent_factors, appliance_df in appliance_struct[appliance_feature].iteritems():
            try:
                region = "Austin"
                df = out_overall[region]
                pred_df = mf_pred_full[appliance][appliance_feature][latent_factors]
                pred_df = pred_df.ix[mf_pred[appliance][appliance_feature][latent_factors].index]
                
                rms = compute_rmse_fraction(appliance, pred_df)

                if rms<best_appliance:

                    best_appliance=rms
                    best_combination[appliance]=(appliance_feature, latent_factors, rms)
            except:
                pass
    result['MF'][appliance]=best_appliance
            
print best_combination

hvac
wm
fridge
mw
oven
dw
{'hvac': ('None', 8, 12.017346591840775), 'wm': ('area_rooms', 2, 1.3309228347386493), 'fridge': ('occ', 6, 3.6224810743222799), 'mw': ('occ_rooms', 4, 0.90966737659949226), 'oven': ('occ', 8, 1.3203534963028163), 'dw': ('occ_area', 2, 0.92017480273612451)}


In [62]:
def get_fhmm(appliance):
    count = 0
    for test_region in ["Austin"]:
        out = {}
        test_df = out_overall[test_region]
        #test_df=test_df[(test_df.full_agg_available==1)&(test_df.md_available==1)]
        
        if appliance=="hvac":
            start, stop=5, 11
        else:
            start, stop=1, 13
        out[appliance] = {}
        test_df_appliance=test_df.ix[test_df[['%s_%d' %(appliance,month) for month in range(start,stop)]].dropna().index]
        for test_home in test_df_appliance.index:
            try:
                #print test_home
                pred_df = pd.read_csv("/Users/nipunbatra/wiki-15min-fhmm-out/%s_%d.csv" %(test_region, test_home),index_col=0)
                pred_df_appliance = pred_df[appliance].ix[start:stop-1]
                gt_df_appliance = test_df.ix[test_home][['%s_%d' %(appliance, month) for month in range(start, stop)]]
                gt_df_appliance.index = pred_df_appliance.index
                error = ((pred_df_appliance-gt_df_appliance).abs()*100/gt_df_appliance)
                #error[error>100]=100
                #acc = 100-error
                out[appliance][test_home]=pred_df_appliance
            except:
                pass
    return pd.DataFrame(out[appliance]).T

In [63]:
lbm_pred_df = pickle.load(open('../data/lbm_15_Austin.pkl'))
def get_lbm(appliance, region):
    if appliance=="hvac":
        start, stop=5, 11
    else:
        start, stop=1, 13
    test_df = out_overall[region]
    #test_df=test_df[(test_df.full_agg_available==1)&(test_df.md_available==1)]
    pred_df = lbm_pred_df[['%s_%d' %(appliance,month) for month in range(start,stop)]]

    test_df_appliance=test_df.ix[test_df[['%s_%d' %(appliance,month) for month in range(start,stop)]].dropna().index]
    gt_df = test_df_appliance[['%s_%d' %(appliance, month) for month in range(start, stop)]]
    gt_df.columns = pred_df.columns
    error = (gt_df-pred_df).abs().div(gt_df).mul(100)
    return pred_df

In [64]:
def get_regional_average(appliance, test_region='Austin'):
    try:
        contri_austin_appliance = contribution[test_region]['Cooling'][appliance]
        if appliance=="hvac":
            start, stop=5, 11
        else:
            start, stop=1, 13
        test_df = out_overall[test_region]
        #test_df=test_df[(test_df.full_agg_available==1)&(test_df.md_available==1)]

        test_df_appliance=test_df.ix[test_df[['%s_%d' %(appliance,month) for month in range(start,stop)]].dropna().index]
        pred_df = test_df_appliance[['aggregate_%d' %month for month in range(start, stop)]]*contri_austin_appliance
        pred_df.columns = ['%s_%d' %(appliance, month) for month in range(start, stop)]
        gt_df = test_df_appliance[['%s_%d' %(appliance, month) for month in range(start, stop)]]
        gt_df.columns = pred_df.columns
        error = (gt_df-pred_df).abs().div(gt_df).mul(100)
        return pred_df
    except:
        return np.NaN

In [65]:
for appliance in knn_results.keys():
    try:
    
        pred_df = get_fhmm(appliance)
        pred_df.columns = [appliance+"_"+str(x) for x in pred_df.columns]
        rmse_fraction = compute_rmse_fraction(appliance, pred_df)


        #rms = np.sqrt(mean_squared_error(gt_df, pred_df))
        result['FHMM'][appliance]=rmse_fraction
        pred_df = get_lbm(appliance,'Austin')
        #pred_df.columns = [appliance+"_"+str(x) for x in pred_df.columns]
        rmse_fraction = compute_rmse_fraction(appliance, pred_df)


        #rms = np.sqrt(mean_squared_error(gt_df, pred_df))
        result['LBM'][appliance]=rmse_fraction

        pred_df = get_regional_average(appliance,'Austin')
        #pred_df.columns = [appliance+"_"+str(x) for x in pred_df.columns]
        rmse_fraction = compute_rmse_fraction(appliance, pred_df)


        #rms = np.sqrt(mean_squared_error(gt_df, pred_df))
        result['RA'][appliance]=rmse_fraction
    except:
        pass

In [66]:
pd.set_option('precision',2)

In [67]:
results_df = pd.DataFrame(result)

In [68]:
map_appliance = {'fridge':'Fridge','dw':'Dish\nwasher','mw':'Microwave','hvac':'HVAC','oven':'Oven','wm':'Washing\nmachine'}

In [69]:
#results_df[results_df>100]=100
#results_df=results_df.fillna(100)
results_df.index = [map_appliance[appliance] for appliance in results_df.index]
#results_df.plot(kind='bar', colors=tableau20blind[::2])

In [70]:
results_df = results_df.ix[['HVAC','Fridge','Oven','Dish\nwasher','Washing\nmachine','Microwave']]

In [76]:
results_df = results_df[['FHMM','LBM','RA','KNN','MF']]

In [77]:
results_df

,FHMM,LBM,RA,KNN,MF
HVAC,15.65,29.37,18.40,11.96,12.02
Fridge,3.90,2.69,4.41,3.38,3.62
Oven,34.00,3.84,NaN,1.49,1.32
Dish\nwasher,13.80,1.74,1.22,1.01,0.92
Washing\nmachine,3.89,13.29,1.40,1.45,1.33
Microwave,5.76,1.01,NaN,0.98,0.91


In [82]:
results_df.to_latex('../results/all-homes.txt')

In [83]:
!cat ../results/all-homes.txt

\begin{tabular}{lrrrrr}
\toprule
{} &   FHMM &    LBM &     RA &    KNN &     MF \\
\midrule
HVAC             &  15.65 &  29.37 &  18.40 &  11.96 &  12.02 \\
Fridge           &   3.90 &   2.69 &   4.41 &   3.38 &   3.62 \\
Oven             &  34.00 &   3.84 &    NaN &   1.49 &   1.32 \\
Dish\textbackslashnwasher     &  13.80 &   1.74 &   1.22 &   1.01 &   0.92 \\
Washing\textbackslashnmachine &   3.89 &  13.29 &   1.40 &   1.45 &   1.33 \\
Microwave        &   5.76 &   1.01 &    NaN &   0.98 &   0.91 \\
\bottomrule
\end{tabular}


In [49]:
best_combination

{'dw': ('occ_area', 2, 0.92017480273612451),
 'fridge': ('occ', 6, 3.6224810743222799),
 'hvac': ('None', 8, 12.017346591840775),
 'mw': ('occ_rooms', 4, 0.90966737659949226),
 'oven': ('occ', 8, 1.3203534963028163),
 'wm': ('area_rooms', 2, 1.3309228347386493)}

In [100]:
for appliance in ['fridge','hvac','dw','mw','wm','oven']:
    mf_best = mf_pred[appliance][best_combination[appliance][0]][best_combination[appliance][1]]
    knn_best = knn_results[appliance]
    mf_best.columns = knn_best.columns
    common_index = np.intersect1d(mf_best.index, knn_best.index)
    mf_best = mf_best.ix[common_index]
    knn_best = knn_best.ix[common_index]
    knn_best_copy = knn_best.copy()
    mf_best_copy = mf_best.copy()
    max_two = (mf_best_copy.where(mf_best_copy > knn_best_copy, knn_best_copy).fillna(mf_best_copy))
    print appliance,100*((-mf_best+knn_best)/max_two).unstack().std()

fridge 57.3978457853
hvac 55.0774075892
dw 51.1713352881
mw 57.3493816955
wm 53.9492074301
oven 55.2763277251


In [ ]:
Calculating win/loss

In [97]:
o = {}
for appliance in ['fridge','hvac','dw','mw','wm','oven']:
    print appliance
    mf_best_pred = mf_pred[appliance][best_combination[appliance][0]][best_combination[appliance][1]]
    knn_best_pred = knn_results[appliance]
    mf_best_pred.columns = knn_best_pred.columns
    common_index = np.intersect1d(mf_best_pred.index, knn_best_pred.index)
    region = "Austin"
    df = out_overall[region]
    pred_df = mf_pred[appliance][appliance_feature][latent_factors]
    gt_df = df[pred_df.columns].ix[pred_df.index]
    mf_best_pred = mf_best_pred.ix[common_index]
    knn_best_pred = knn_best_pred.ix[common_index]
    error_mf = (gt_df-mf_best_pred).abs()
    error_knn = (gt_df-knn_best_pred).abs()
    win = (error_knn>error_mf).sum().sum()
    loss = (error_knn<error_mf).sum().sum()
   
    o[appliance]={'win':win, "loss":loss, "win/loss":win*1./loss,"win/(win_loss)":win*1./(win+loss)} 

fridge
hvac
dw
mw
wm
oven


In [98]:
pd.DataFrame(o).T

,loss,win,win/(win_loss),win/loss
dw,288,408,0.586207,1.416667
fridge,423,393,0.481618,0.929078
hvac,277,293,0.514035,1.057762
mw,249,315,0.558511,1.265060
oven,179,181,0.502778,1.011173
wm,358,338,0.485632,0.944134


In [92]:
knn_best_pred.columns, mf_best_pred.columns

(Index([u'hvac_5', u'hvac_6', u'hvac_7', u'hvac_8', u'hvac_9', u'hvac_10'], dtype='object'),
 Index([u'hvac_5', u'hvac_6', u'hvac_7', u'hvac_8', u'hvac_9', u'hvac_10'], dtype='object'))

In [118]:
knn_best.median().mean()

51.70860070119945

In [119]:
appliance

'oven'

In [112]:
a=(gt_df>10).sum(axis=1)
a[a<12]

59       0
457     11
499     11
954     11
2907    11
3510    10
3893     0
4856    11
5718    11
8419     0
dtype: int64